In [ ]:
import mne
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
import os
import numpy as np
import pandas as pd
from glob import glob

In [ ]:
ds_root = "D:/yikang1020/yikangsystem1020/2_Projects/复杂知觉决策的证据积累模型：EEG和行为的证据/2_Study/2_3_RawData/RawData/Ostwald_OSF/BIDS_data"
anlz_dir = "/home/jovyan/hddm/2_Study/2_4_Analysis/2_4_3_tmp_data/filtered_subj"
out_MR = "sourcedata-eeg_outside-MRT/eeg"

In [ ]:
hc = ['Stimulus/S 10',#high phase coherence , prioritized     [l]
 'Stimulus/S 11',#high phase coherence , prioritized     [r]
 'Stimulus/S 20',#high phase coherence , not prioritized [l]
 'Stimulus/S 21']#low phase coherence  , not prioritized [r]
lc = ['Stimulus/S 30',#low phase coherence  , prioritized     [l]
 'Stimulus/S 31',#low phase coherence  , prioritized     [r]
 'Stimulus/S 40',#low phase coherence  , not prioritized [l]
 'Stimulus/S 41']

In [ ]:
list_out_subj = ['sub-001', 
                 'sub-003', 
                 'sub-004', 
                 'sub-005', 
                 'sub-006', 
                 'sub-008', 
                 'sub-009', 
                 'sub-010', 
                 'sub-012',
                 'sub-013',
                 'sub-014',
                 'sub-015', 
                 'sub-016',
                 'sub-017']

In [ ]:
for subject in list_out_subj:
    subj = subject
    folder_MR = out_MR 
    src_dir_subj = os.path.join(ds_root,subj,folder_MR)
    fname = glob(src_dir_subj + "/*.vhdr")[0]
    
    raw = mne.io.read_raw_brainvision(fname, preload = True)
    
    raw.set_channel_types({'EOG':'eog'})   
    raw.set_channel_types({'ECG':'ecg'})
    
    raw.resample(256, npad="auto")
    
    raw.filter(1, 30, fir_design='firwin', picks=['eeg'])
    
    raw.set_eeg_reference('average', projection=True).apply_proj()
    
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    
    event_dict ={'Stimulus/S  5': 5,
 'Stimulus/S  6': 6,
 'Stimulus/S 10': 10,#high phase coherence , prioritized     [l]
 'Stimulus/S 11': 11,#high phase coherence , prioritized     [r]
 'Stimulus/S 20': 20,#high phase coherence , not prioritized [l]
 'Stimulus/S 21': 21,#high phase coherence , not prioritized [r]
 'Stimulus/S 30': 30,#low phase coherence  , prioritized     [l]
 'Stimulus/S 31': 31,#low phase coherence  , prioritized     [r]
 'Stimulus/S 40': 40,#low phase coherence  , not prioritized [l]
 'Stimulus/S 41': 41}#low phase coherence  , not prioritized [r]
    
    reject_criteria = dict(eeg=100e-6) 
    
    epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict, tmin=-0.1, tmax=0.8,
                    reject=reject_criteria, baseline = (None,0), preload=True, picks=['eeg'])
    
    ica = mne.preprocessing.ICA(n_components=50, random_state=97, method='fastica')
    ica.fit(epochs) 
    
    ica.exclude = []                                   
    eog_indices, eog_scores = ica.find_bads_eog(raw) 
                                                                         
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='ctps')
                                                                   
    ica.exclude = eog_indices + ecg_indices 
    
    ica.apply(epochs) 
    
    tname = os.path.join("D:/yikang1020/yikangsystem1020/2_Projects/复杂知觉决策的证据积累模型：EEG和行为的证据/2_Study/2_4_Analysis/2_4_3_tmp_data/eeg2", subj + "_pred.fif")
    
    epochs.save(tname,overwrite=True)
    
    print(subj+'has done')